# data는 e9t(Lucy Park)님께서 github에 공유해주신 네이버 영화평점 데이터를 사용하였습니다.
# https://github.com/e9t/nsmc

In [1]:
# data를 읽어오는 함수
def read_txt(path_to_file):
    txt_ls = []
    label_ls = []

    with open(path_to_file) as f:
        for i, line in enumerate(f.readlines()[1:]):
            _, txt, label = line.split('\t')
            txt_ls.append(txt)
            label_ls.append(int(label.replace('\n','')))

    return txt_ls, label_ls

In [2]:
x_train, y_train = read_txt('../ratings_train.txt')
x_test, y_test = read_txt('../ratings_test.txt')

In [3]:
len(x_train), len(y_train), len(x_test), len(y_test)

(150000, 150000, 50000, 50000)

In [4]:
x_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

## 띄어쓰기로 구분

In [5]:
x_train = [x.split() for x in x_train]
x_test = [x.split() for x in x_test]

In [6]:
x_train[0]

['아', '더빙..', '진짜', '짜증나네요', '목소리']

## 고유 토큰 인덱싱

In [7]:
from collections import defaultdict

In [8]:
# 단어마다 고유한 인덱스를 부여하기 위한 dictionary
token_to_index = defaultdict(lambda : len(token_to_index))

In [9]:
# 단어에 대한 고유 인덱스를 부여하는 함수
def convert_token_to_idx(token_ls):
    for tokens in token_ls:
        yield [token_to_index[token] for token in tokens]
    return

In [10]:
x_train = list(convert_token_to_idx(x_train))
x_test = list(convert_token_to_idx(x_test))

In [11]:
# 고유 인덱스로 변환될 경우, 원래 어떤 단어였는지 알기 어려우므로,
# 인덱스로 변환된 단어를 본래의 단어로 재변환하기 위한 dictionary 생성
index_to_token = {val : key for key,val in token_to_index.items()}

#### 인덱싱 결과 확인 

In [12]:
import operator

In [13]:
for k,v in sorted(token_to_index.items(), key=operator.itemgetter(1))[:5]:
    print (k,v)

아 0
더빙.. 1
진짜 2
짜증나네요 3
목소리 4


In [14]:
x_train[0]

[0, 1, 2, 3, 4]

### 빈(empty) 단어 가방(Bag of Words) 생성

In [15]:
n_train_reviews = len(x_train)       # 학습용 리뷰의 총 수
n_test_reviews = len(x_test)
n_unique_word = len(token_to_index)  # 고유 단어의 갯수 (BOW의 차원의 크기) 

In [16]:
n_unique_word

450541

### numpy를 사용하면 memory error 발생 

In [17]:
import numpy as np

In [18]:
bow = np.zeros((n_train_reviews, n_unique_word), dtype=np.int8)

MemoryError: 

### Scipy 패키지 활용

In [20]:
import scipy.sparse as sps

In [21]:
# 학습용 리뷰 수(150,000) x 고유 단어의 수(450,541)의 크기를 갖는 빈 단어가방 생성
bow_train = sps.lil_matrix((n_train_reviews, n_unique_word), dtype=np.int8)
bow_test = sps.lil_matrix((n_test_reviews, n_unique_word), dtype=np.int8)

### 단어 가방 채우기

In [22]:
for i, tokens in enumerate(x_train):
    for token in tokens:
        # i번 째 리뷰에 등장한 단어들을 세서, 고유 번호에 1씩 더해준다.
        bow_train[i, token] += 1

In [23]:
for i, tokens in enumerate(x_test):
    for token in tokens:
        # i번 째 리뷰에 등장한 단어들을 세서, 고유 번호에 1씩 더해준다.
        bow_test[i, token] += 1

## Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

In [48]:
model = LogisticRegression()

## Train

In [49]:
model.fit(bow_train, y_train)

/home/donghyungko/anaconda3/envs/fininsight_python3.5/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

# Test

In [50]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [51]:
predict = model.predict(bow_test)
accuracy = accuracy_score(y_test, predict)

In [52]:
print('Accuracy : ',accuracy)
print(classification_report(y_test, predict))

Accuracy :  0.7906
              precision    recall  f1-score   support

           0       0.77      0.83      0.80     24827
           1       0.82      0.75      0.78     25173

   micro avg       0.79      0.79      0.79     50000
   macro avg       0.79      0.79      0.79     50000
weighted avg       0.79      0.79      0.79     50000

